In [1]:
import uiautomation as auto
import subprocess
from robot.libraries.BuiltIn import *

"""
Huy, 13/6/2021
"""

auto.uiautomation.OPERATION_WAIT_TIME = 1
auto.uiautomation.TIME_OUT_SECOND = 15
auto.uiautomation.SEARCH_INTERVAL = 1

# todo System variables
ie_window: auto.WindowControl = None
multimedia_window: auto.WindowControl = None


def safe_click(element, sleepTime: float = 1, simulateMove: bool = False, waitTime: float = 0.5, focus: bool = False):
    if focus:
        element.SetFocus()
    element.MoveCursorToMyCenter()
    time.sleep(sleepTime)
    element.Click(simulateMove=simulateMove)
    time.sleep(waitTime)


def find_and_click(element, maxSearchTime: float = 3, searchInterval: float = 1, sleepTime: float = 1,
                   waitTime: float = 0.5, simulateMove: bool = False, focus: bool = False):
    if element.Exists(maxSearchTime, searchInterval):
        safe_click(element, sleepTime, simulateMove, waitTime, focus)
        return True
    return False

In [46]:
def _open_global_ie_window():
    global ie_window
    ie_window = None
    print("global ie_window = ", ie_window)
    ie_path = r"C:\\Program Files\\Internet Explorer\\iexplore.exe"
    if "iexplore.exe" not in str(subprocess.run(['tasklist'], capture_output=True, text=True).stdout):
        print("iexplore.exe not in tasklist, subprocess.Popen(ie_path)")
        subprocess.Popen(ie_path)
        time.sleep(3)
    ie_window = auto.WindowControl(searchDepth=1, ClassName="IEFrame")
    if not ie_window.Exists(10, 1):
        print("ie not exists")
        BuiltIn().log_to_console("Cannot open IE")
        BuiltIn().fail("Fail to open IE")
    i = 3  # number of find
    time.sleep(2)
    while i > 0:
        print(i)
        if not ie_window.SetActive():
            ie_window.SetFocus()
        # find and close asks default
        a1=find_and_click(ie_window.WindowControl(searchDepth=1, ClassName="#32770").ButtonControl(Name="Close"), 1, 0.2,
                       0.5, 1)
        # find and close Notify bar
        a2=find_and_click(
            ie_window.PaneControl(searchDepth=1, ClassName="Frame Notification Bar").ButtonControl(Name="Close"), 1, 0.2,
            0.5, 1)
        time.sleep(1)
        if a1 or a2:
            print(a1, a2)
            i = 3
            continue
        i -= 1

In [48]:
_open_global_ie_window()

global ie_window =  None
iexplore.exe not in tasklist, subprocess.Popen(ie_path)
3
True False
3
2
1


In [55]:
def ie_open_data_management():
    global ie_window
    time.sleep(3)
    find_and_click(ie_window.PaneControl(ClassName="SSTreeWndClass"), 5, 1, 1, 2, focus=True)
    pane_control = ie_window.PaneControl(ClassName="SSTreeWndClass")
    print(pane_control.SetFocus(), pane_control)
    pane_control.SendKeys("{down 2}", waitTime=1, interval=0.05)
    pane_control.SendKeys("{space}", waitTime=1, interval=0.05)
    find_and_click(ie_window.ButtonControl(AutomationId="btnLaunch"), 5, 1, 0.5, 1, focus=True)
    i = 3
    while i > 0:
        print(i)
        # find and click Open button in notify bar
        # find and click OK button if an instance Data management is already running
        # find and click YES button if user already logged in to Data management
        a1 = find_and_click(ie_window.PaneControl(searchDepth=1, ClassName="Frame Notification Bar").ButtonControl(Name="Open"),2, 0.5, 0.5, 2, focus=True)
        a2 = find_and_click(auto.WindowControl(searchDepth=1, AutomationId="TrustManagerPromptUI").ButtonControl(AutomationId="btnInstall"), 2,0.5,0.5,10,True)
        a3 = find_and_click(auto.WindowControl(searchDepth=1, Name="CCMM Data Management").ButtonControl(Name="OK"), 2, 0.5, 0.5,1, focus=True)
        a4 = find_and_click(auto.WindowControl(searchDepth=1, Name="Data Management").ButtonControl(Name="Yes"), 2, 0.5, 0.5,1, focus=True)
        if a1 or a2 or a3 or a4:
            print(a1, a2, a3, a4)
            i = 3
            continue
        i -= 1

In [58]:
ie_open_data_management()

True ControlType: PaneControl    ClassName: SSTreeWndClass    AutomationId:     Rect: (746,176,893,604)[147x428]    Name:     Handle: 0x180888(1575048)
3
True False True False
3
2
1


In [37]:
import threading
lock = threading.Lock()

In [38]:
lock

<unlocked _thread.lock object at 0x00000168A4919750>

In [41]:
def a1():
    global auto
    ie_address = auto.WindowControl(searchDepth=1, ClassName="IEFrame").PaneControl(ClassName="Address Band Root")
    if ie_address.Exists(3, 1):
        ie_address.SetFocus()
        ie_address.Click(simulateMove=False)


def a2():
    global auto
    lock.acquire()
    ie_tab = auto.WindowControl(searchDepth=1, ClassName="IEFrame").PaneControl(ClassName="TabBandClass")
    if ie_tab.Exists(3, 1):
        ie_tab.SetFocus()
        ie_tab.Click()
    lock.release()


def runner():
    t1 = threading.Thread(target=a1)
    t2 = threading.Thread(target=a2)
    t1.start()
    t2.start()
    print("start t1, t2")
    t1.join()
    t2.join()
    print("Done")

In [43]:
runner()

start t1, t2
Done
